In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField , StructType , StringType , IntegerType , ArrayType , LongType
from pyspark.sql.functions import from_json , col , explode



In [2]:
spark = (
    SparkSession.builder
    .appName("Streaming from Kafka")
    .config("spark.streaming.stopGracefullyOnShutdown", True)
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.1")
    .config("spark.jars" , '/home/mhai/Pineline-Realtime-Streaming/postgresql-42.7.3.jar')
    .config("spark.sql.shuffle.partitions", "5")
    .master("local[*]")
    .getOrCreate()
)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/29 18:15:11 WARN Utils: Your hostname, Admin-PC, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/29 18:15:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/mhai/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/mhai/.ivy2.5.2/cache
The jars for the packages stored in: /home/mhai/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-677ea044-5755-4d0f-98e4-bc7600dfc9bc;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.s

In [3]:
spark

In [4]:
print(spark.sparkContext._jsc.sc().listJars())


List(spark://10.255.255.254:33093/jars/postgresql-42.7.3.jar)


In [5]:
device = spark.readStream.format('kafka') \
                   .option('kafka.bootstrap.servers' , '172.23.152.231:9092') \
                   .option('subscribe' , 'device-data') \
                   .option('startingOffsets' , 'earliest') \
                   .load()

In [6]:
device.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
device = device.withColumn('value' , device['value'].cast('string'))

In [8]:
device.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [9]:
device_schema = StructType(
    [
        StructField('customerId' , StringType() , True) ,
        StructField('data' , StructType(
            [StructField('devices' ,
                ArrayType(StructType([
                    StructField('deviceId' , StringType() , True) ,
                    StructField('measure' , StringType() , True) ,
                    StructField('status' , StringType() , True) ,
                    StructField('temperature' , StringType() , True)
                ]),True ) , True ) 
            ]) , True) ,
        StructField('eventId' , StringType() , True) ,
        StructField('eventOffset' , LongType() , True) ,
        StructField('eventPublisher' , StringType() , True) ,
        StructField('eventTime' , StringType(), True )          
    ]
)

In [10]:
device = device.withColumn('value_json' , from_json(col('value') , device_schema)).select('value_json.*')

In [11]:
device_explode = device.withColumn('devices' , explode(col('data.devices')))

In [12]:
device = device_explode.withColumn('deviceId', col('devices.deviceId')) \
                        .withColumn('measure', col('devices.measure')) \
                        .withColumn('status', col('devices.status')) \
                        .withColumn('temperature', col('devices.temperature')) \
                        .drop('data') \
                        .drop('devices')

In [13]:
def save_psql(data , batch_id) :
    print('batch id : ' , str(batch_id))    
    data.write.format('jdbc') \
            .mode('append') \
            .option('driver' , 'org.postgresql.Driver') \
            .option('url' , 'jdbc:postgresql://192.168.1.5:5432/test') \
            .option('dbtable' , 'public.device_data') \
            .option('user' , 'postgres') \
            .option('password' , 'MINHHAI123') \
            .save()
    data.show()
    
    

In [14]:
device.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)
 |-- temperature: string (nullable = true)



In [ ]:
device.writeStream \
        .foreachBatch(save_psql) \
        .outputMode('append') \
        .trigger(processingTime = '10 seconds') \
        .option('checkpointLocation' , 'checkpoint_dir_kafka') \
        .start() \
        .awaitTermination() 

25/10/29 18:15:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


batch id :  32
+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure| status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+
|   CI00101|1450324a-c546-417...|      10038|        device|2023-01-05 11:13:...|    D004|      C|SUCCESS|         20|
|   CI00101|1450324a-c546-417...|      10038|        device|2023-01-05 11:13:...|    D004|      C|SUCCESS|          1|
|   CI00101|1450324a-c546-417...|      10038|        device|2023-01-05 11:13:...|    D002|      C|SUCCESS|         21|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+

batch id :  33


+----------+--------------------+-----------+--------------+--------------------+--------+-------+------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+------+-----------+
|   CI00290|b372c138-26b4-4e9...|      10000|        device|2025-10-29 18:17:...|    D002|      C| ERROR|         18|
|   CI00290|b372c138-26b4-4e9...|      10000|        device|2025-10-29 18:17:...|    D001|      C| ERROR|         14|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+------+-----------+

batch id :  34
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+---

+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00174|23ce36cd-cd0b-44f...|      10014|        device|2025-10-29 18:18:...|    D007|      C|   ERROR|          4|
|   CI00174|23ce36cd-cd0b-44f...|      10014|        device|2025-10-29 18:18:...|    D007|      C| SUCCESS|         35|
|   CI00174|23ce36cd-cd0b-44f...|      10014|        device|2025-10-29 18:18:...|    D008|      C|STRANDBY|         21|
|   CI00174|23ce36cd-cd0b-44f...|      10014|        device|2025-10-29 18:18:...|    D008|      C|   ERROR|         22|
|   CI00120|476bf931-6c0b-4b5...|      10015|        device|2025-10-29 18:18:...|    NULL|      C|STRANDBY|         42|
|   CI00226|9d196709-51ca-4b8...|      1

+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00252|7d63f45d-7c87-4dc...|      10018|        device|2025-10-29 18:18:...|    NULL|      C|STRANDBY|         42|
|   CI00252|7d63f45d-7c87-4dc...|      10018|        device|2025-10-29 18:18:...|    D002|      C|STRANDBY|         13|
|   CI00252|7d63f45d-7c87-4dc...|      10018|        device|2025-10-29 18:18:...|    D002|      C|STRANDBY|         50|
|   CI00252|7d63f45d-7c87-4dc...|      10018|        device|2025-10-29 18:18:...|    NULL|      C|   ERROR|         34|
|   CI00252|7d63f45d-7c87-4dc...|      10018|        device|2025-10-29 18:18:...|    D005|      C|STRANDBY|         17|
|   CI00284|b44835ca-750e-48b...|      1

...

+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00291|1b1b3904-19fc-428...|      11053|        device|2025-10-29 19:06:...|    D005|      C| SUCCESS|          3|
|   CI00291|1b1b3904-19fc-428...|      11053|        device|2025-10-29 19:06:...|    D008|      C|    NULL|         30|
|   CI00291|1b1b3904-19fc-428...|      11053|        device|2025-10-29 19:06:...|    D006|      C|STRANDBY|          9|
|   CI00291|1b1b3904-19fc-428...|      11053|        device|2025-10-29 19:06:...|    D003|      C|STRANDBY|         25|
|   CI00291|1b1b3904-19fc-428...|      11053|        device|2025-10-29 19:06:...|    D002|      C|    NULL|         29|
|   CI00291|1b1b3904-19fc-428...|      1

+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00130|c55d49b1-916a-495...|      11076|        device|2025-10-29 19:06:...|    D008|      C| SUCCESS|         34|
|   CI00130|c55d49b1-916a-495...|      11076|        device|2025-10-29 19:06:...|    D007|      C|STRANDBY|         36|
|   CI00130|c55d49b1-916a-495...|      11076|        device|2025-10-29 19:06:...|    NULL|      C|    NULL|         34|
|   CI00130|c55d49b1-916a-495...|      11076|        device|2025-10-29 19:06:...|    D001|      C| SUCCESS|         45|
|   CI00130|c55d49b1-916a-495...|      11076|        device|2025-10-29 19:06:...|    D007|      C|STRANDBY|         12|
|   CI00210|bf86a383-0976-4dc...|      1

+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00134|25e15648-55f5-4d2...|      11082|        device|2025-10-29 19:06:...|    NULL|      C|   ERROR|         45|
|   CI00134|25e15648-55f5-4d2...|      11082|        device|2025-10-29 19:06:...|    D008|      C|   ERROR|         48|
|   CI00134|25e15648-55f5-4d2...|      11082|        device|2025-10-29 19:06:...|    D007|      C|    NULL|         28|
|   CI00134|25e15648-55f5-4d2...|      11082|        device|2025-10-29 19:06:...|    D006|      C|    NULL|         48|
|   CI00134|25e15648-55f5-4d2...|      11082|        device|2025-10-29 19:06:...|    D001|      C|   ERROR|         27|
|   CI00134|25e15648-55f5-4d2...|      1

+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00147|f326c810-0ff0-4a2...|      11090|        device|2025-10-29 19:07:...|    NULL|      C| SUCCESS|          0|
|   CI00147|f326c810-0ff0-4a2...|      11090|        device|2025-10-29 19:07:...|    D002|      C|   ERROR|         49|
|   CI00147|f326c810-0ff0-4a2...|      11090|        device|2025-10-29 19:07:...|    D006|      C|   ERROR|         28|
|   CI00147|f326c810-0ff0-4a2...|      11090|        device|2025-10-29 19:07:...|    D002|      C|STRANDBY|         46|
|   CI00216|34ca2ba2-d767-499...|      11091|        device|2025-10-29 19:07:...|    D002|      C|STRANDBY|         39|
|   CI00216|34ca2ba2-d767-499...|      1

+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00183|85d2f781-1cc0-471...|      11096|        device|2025-10-29 19:07:...|    D002|      C|   ERROR|         39|
|   CI00148|4b9eb36c-eb51-4d3...|      11097|        device|2025-10-29 19:07:...|    D003|      C|STRANDBY|         14|
|   CI00148|4b9eb36c-eb51-4d3...|      11097|        device|2025-10-29 19:07:...|    D004|      C|STRANDBY|         10|
|   CI00148|4b9eb36c-eb51-4d3...|      11097|        device|2025-10-29 19:07:...|    D004|      C|   ERROR|         36|
|   CI00148|4b9eb36c-eb51-4d3...|      11097|        device|2025-10-29 19:07:...|    D004|      C| SUCCESS|         30|
|   CI00148|4b9eb36c-eb51-4d3...|      1